In [5]:
# imports
import requests
from IPython.display import JSON 
import pandas as pd
import numpy as np
import os

# Foursquare

In [25]:
fsq_api_key = os.environ.get('fsq3zaWaSmh1ImUoq/3I38e1V6ldOZbI4DahLfbRfbQz3zM=')

In [7]:
city_bikes_marseille = requests.get("http://api.citybik.es/v2/networks/le-velo").json()
JSON(city_bikes_marseille)

<IPython.core.display.JSON object>

In [8]:
df_city_bikes_marseille = pd.read_json("http://api.citybik.es/v2/networks/le-velo")

In [9]:
stations_archive = df_city_bikes_marseille["network"]["stations"]

In [22]:
def final_array_of_parsed_stations ():
    array_of_stations = []
    for stations in stations_archive:
        name = stations["name"]
        latitude = stations["latitude"]
        longitude = stations["longitude"]
        empty_slots = stations["empty_slots"]
        Available_bikes = stations["free_bikes"]
        timestamp = stations["timestamp"]
        parsed_array_of_stations = {"name": name,"Latitude": latitude, "Longitude": longitude, "Empty Slots": empty_slots, "free_bikes": Available_bikes, "timestamp": timestamp}
        array_of_stations.append(parsed_array_of_stations)
    return array_of_stations

marseille_bike_stations = final_array_of_parsed_stations()
df_array_of_stations = pd.DataFrame(final_array_of_parsed_stations())

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [26]:

import requests

def fsq_req_marseille(bike_stations, fsq_api_key): 
    radius = 1000
    fsq_data = []

    for station in bike_stations:
        latitude = station["latitude"]
        longitude = station["longitude"]
        foursquare_url = f"https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}"

        headers = {
            "accept": "application/json",
            "Authorization": f"Bearer {fsq_api_key}"
        }

        response = requests.get(foursquare_url, headers=headers)

        if response.status_code == 200:
            fsq_data.append(response.json())
        else:
            print(f"API request for station failed with status code:", response.status_code)

    return fsq_data





Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating